In [ ]:
//#r "B:\BoSSS-smuda\public\src\L4-application\BoSSSpad\bin\Release\net5.0\BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Solution;
using BoSSS.Application.XNSE_Solver;
using System.IO;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution.Gnuplot;
using BoSSS.Solution.Control;

using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [ ]:
// Is used at Jenkins to generate individual names (for output .json)
string dbname = System.Environment.GetEnvironmentVariable("DATABASE_NAME");
string buildname = System.Environment.GetEnvironmentVariable("JOB_NAME");
//defaults
buildname = String.IsNullOrEmpty(buildname)? "Benchmark" : buildname;
//string thedate = $"{System.DateTime.Today.Day}-{System.DateTime.Today.Month}-{System.DateTime.Today.Year}";
dbname = String.IsNullOrEmpty(dbname)? "dbname" : dbname;
string table_name = String.Concat(buildname, "_", dbname);

In [ ]:
//string table_name = @"SolverRuns--2020-10-23";

In [ ]:
string CurrentDocDir = Directory.GetCurrentDirectory();
string path = String.Concat(CurrentDocDir,@"/",table_name,".json");
var SubTab = TableExtensions.LoadFromFile(path);

In [ ]:
//extract all Degrees and put them into array
int rowcount          = SubTab.Rows.Count;
string selectedcolumn = "DGdegree:T";
int[] tmp= new int[20];

int count = 0;
for(int r=0;r<rowcount;r++){
System.Data.DataRow row = (System.Data.DataRow)SubTab.Rows[r];
int bla                 = Convert.ToInt32(row[selectedcolumn]);
if (tmp[bla]==0){
  tmp[bla]=1;
  count++;
}
}
int[] DGdegrees = new int[count];
int count2 = 0;
for(int i=0;i<tmp.Length;i++){
   if(tmp[i]==1){
      DGdegrees[count2]=i;
      count2++; 
   }
}
DGdegrees

index,value
0,2
1,3
2,5


In [ ]:
// downward compatibility ...
string LSolverCode = SubTab.Columns.Contains("LinearSolver.SolverCode")? "LinearSolver.SolverCode" : "LinearSolverCode";

In [ ]:
using BoSSS.Solution.AdvancedSolvers;

In [ ]:
int rows    = 4;
int columns = 1;
//LinearSolverCode[] ignore_solvers = {LinearSolverCode.direct_pardiso, LinearSolverCode.classic_cg, LinearSolverCode.exp_softpcg_schwarz, LinearSolverConfig.Code.exp_direct_lapack};
LinearSolverCode [] ignore_solvers ={};
Plot2Ddata[,] multiplots = new Plot2Ddata[rows,columns];
int pDegree = 0;
for(int iRow = 0; iRow < rows; iRow++) {
for(int iCol = 0; iCol < columns; iCol++) {
    
    if(pDegree > rows*columns-1)
        continue;
    int tmpDG = -1;
    if(pDegree < DGdegrees.Length)
        tmpDG = DGdegrees[pDegree];
    
    //Create Graphs
    multiplots[iRow,iCol] = SubTab.ToPlot("DOFs", "minSolRunT", // column for x- and y
       delegate (int iTabRow, 
                 IDictionary<string, object> Row, 
                 out string Nmn, 
                 out PlotFormat Fmt) { 
           // - - - - - - - - - - - - - - - - - - - - - - - - 
           // PlotRowSelector:
           // selects, which table row goes to which graph,
           // and the respective color  
           // - - - - - - - - - - - - - - - - - - - - - - - - 
           int k = Convert.ToInt32(Row["DGdegree:T"]);
           if(k != tmpDG) {
                // degree does not match -> not in this plot
                Nmn = null;
                Fmt = null;
                return;
           }
 
           LinearSolverCode solver_name = (LinearSolverCode)Convert.ToInt32(Row["LinearSolver.SolverCode"]);
           //ignore the solvers specified in ingore_solvers
           foreach(LinearSolverCode sc in ignore_solvers){
               if(solver_name==sc){
                   System.Console.WriteLine("skipped");
                   Nmn = null;
                   Fmt = null;
               return;
               }
           }
           
           // ===!!!=== CAUTION ===!!!===
           // Solverframework (SolverEnum numbers) have changed! Please check LinearSolver and NonLinearSolver for details
           
           //process the other solvers
           Fmt = new PlotFormat();
           switch(solver_name) {
           case LinearSolverCode.direct_pardiso: 
               Nmn           = "Pardiso"; 
               Fmt.PointType = PointTypes.OpenCircle;
               Fmt.DashType  = DashTypes.Dotted;
               break;
           case LinearSolverCode.exp_gmres_levelpmg: 
               Nmn           = "GMRES w. pTG"; 
               Fmt.PointType = PointTypes.Box;
               break;
           case LinearSolverCode.exp_Kcycle_schwarz: 
               Nmn           = "Kcycle w. add.-Schwarz"; 
               Fmt.PointType = PointTypes.LowerTriangle;
               break;
           default:
               Console.WriteLine("unknown: " + solver_name); 
               Nmn = "unknown";
               break;
           } 
           Console.WriteLine("name is: " + solver_name); 
           Fmt.PointSize = 0.5;
           Fmt.Style     = Styles.LinesPoints;
           Fmt.LineColor = LineColors.Black;
           Fmt.LineWidth = 3;
       });
    double[] dof = new[]{1e3,1e6};
    double[] linT = dof.Select(x => x*0.001).ToArray();
    var linP = new Plot2Ddata.XYvalues("linear", dof, linT);
    linP.Format.FromString("- black");
    ArrayTools.AddToArray(linP, ref multiplots[iRow,iCol].dataGroups);
 
    //all about axis
    string Title = string.Format("$k = {0}$", tmpDG);
    //multiplots[iRow,iCol].Ylabel = Title;
    multiplots[iRow,iCol].Title = Title;
    multiplots[iRow,iCol].LogX = true;
    multiplots[iRow,iCol].LogY = true;
    //specify range of axis
    multiplots[iRow,iCol].YrangeMin = 1e-2;
    multiplots[iRow,iCol].YrangeMax = 1e+4;
    multiplots[iRow,iCol].XrangeMin = 1e1;
    multiplots[iRow,iCol].XrangeMax = 1e7;
    //multiplots[iRow,iCol].Y2rangeMin = 1e-3;
    //multiplots[iRow,iCol].Y2rangeMax = 1e+4;
    //multiplots[iRow,iCol].X2rangeMin = 1e2;
    //multiplots[iRow,iCol].X2rangeMax = 1e7;
    
    //spacing around plots
    multiplots[iRow,iCol].ShowLegend = false;
    multiplots[iRow,iCol].tmargin = 0;
    multiplots[iRow,iCol].bmargin = 1.5;
    multiplots[iRow,iCol].lmargin = 10;
    multiplots[iRow,iCol].rmargin = 5;
    multiplots[iRow,iCol].ShowXtics=false;

    //I am legend ...
    if(iRow == 0){
        multiplots[iRow,iCol].ShowLegend = true;
        multiplots[iRow,iCol].LegendAlignment=new string[]{"i","t","l"};
        multiplots[iRow,iCol].LegendSwap=true;
    }

    //and i am special ...
    if(iRow == rows-2){
        multiplots[iRow,iCol].ShowXtics=true;
        multiplots[iRow,iCol].Xlabel = "DOF";
    }

    //dummy plot
    if(iRow == rows-1){
        multiplots[iRow,iCol] = new Plot2Ddata();
    }

    pDegree++;
}                        
}
//multiplots.AddDummyPlotsForLegend()
multiplots.PlotNow()

name is: exp_gmres_levelpmg
name is: exp_Kcycle_schwarz
name is: direct_pardiso
name is: exp_gmres_levelpmg
name is: exp_Kcycle_schwarz
name is: exp_gmres_levelpmg
name is: exp_Kcycle_schwarz
name is: exp_gmres_levelpmg
name is: exp_Kcycle_schwarz
name is: exp_gmres_levelpmg
name is: exp_Kcycle_schwarz
name is: direct_pardiso
name is: direct_pardiso
name is: direct_pardiso
name is: exp_Kcycle_schwarz
name is: exp_gmres_levelpmg
name is: exp_Kcycle_schwarz
name is: exp_gmres_levelpmg
name is: direct_pardiso
name is: exp_gmres_levelpmg
name is: exp_Kcycle_schwarz
name is: exp_gmres_levelpmg
name is: exp_gmres_levelpmg
name is: exp_Kcycle_schwarz
name is: exp_Kcycle_schwarz
name is: direct_pardiso
name is: direct_pardiso
name is: exp_gmres_levelpmg
name is: exp_Kcycle_schwarz
name is: exp_gmres_levelpmg
name is: exp_gmres_levelpmg
name is: exp_Kcycle_schwarz
name is: direct_pardiso
name is: exp_gmres_levelpmg
name is: exp_Kcycle_schwarz
name is: direct_pardiso
Using gnuplot: B:\BoSSS-BIN\

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 3 $ 
 
 
 
 
 $10 4 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 GMRES w. pTG 
 
 
 
 
 GMRES w. pTG 
 
 
 
 
 
 
 
 
 
 
 
 Kcycle w. add.-Schwarz 
 
 
 Kcycle w. add.-Schwarz 
 
 
 
 
 
 
 
 
 
 
 
 Pardiso 
 
 
 Pardiso 
 
 
 
 
 
 
 
 
 
 
 linear 
 
 
 
 
 linear 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 3 $ 
 
 
 
 
 $10 4 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $k = 3$ 
 
 
 Kcycle w. add.-Schwarz 
 
 
 
 
 
 
 
 
 
 
 GMRES w. pTG 
 
 
 
 
 
 
 
 
 Pardiso 
 
 
 
 
 
 
 linear 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 3 $ 
 
 
 
 
 $10 4 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 3 $ 
 
 
 
 
 $10 4 $ 
 
 
 
 
 $10 5 $ 
 
 
 
 
 $10 6 $ 
 
 
 
 
 $10 7 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 DOF 
 
 
 
 
 $k = 5$ 
 
 
 GMRES w. pTG 
 
 
 
 
 
 
 
 
 
 Kcycle w. add.-Schwarz 
 
 
 
 
 
 
 Pardiso 
 
 
 
 
 
 linear

In [ ]:
string dirpath2 = Path.Combine(CurrentDocDir, "data");
if(!System.IO.Directory.Exists(dirpath2)){
    System.IO.Directory.CreateDirectory(dirpath2);
}

In [ ]:
//Plot2Ddata(dataGroups.ToArray()).WithLogX().WithLogY()
var k_sweep = DGdegrees;
var names = multiplots[0,0].dataGroups.Select(s=>s.Name);
int iPlot = 0;
foreach(var plot in multiplots){
string sweep = plot.Title;
var datagroups = plot.dataGroups;

foreach(var group in datagroups){
    string name = group.Name;
    string shortname = name.Replace('.','_').Replace(' ','_');
    string filepath = Path.Combine(dirpath2,$"{k_sweep[iPlot]}"+"_"+shortname);
    plot.Extract(name).SaveToTextFile(filepath);
}
iPlot++;
}

In [ ]:
var CL = multiplots.ToGnuplot().PlotCairolatex(xSize:14,ySize:12);

Using gnuplot: B:\BoSSS-BIN\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" inside top left Left reverse 


In [ ]:
string dirpath = Path.Combine(CurrentDocDir, "plots");
if(!System.IO.Directory.Exists(dirpath)){
    System.IO.Directory.CreateDirectory(dirpath);
}

In [ ]:
CL.WriteMinimalCompileableExample(
    Path.Combine(CurrentDocDir, "plots", "MainConstCoeffPoissonScaling.tex"),
    "ConstCoeffPoissonScaling.tex");